# Presentation of new pyALF features

**1.** Import `Simulation` class from the `py_alf` python module, which provides the interface with ALF:

In [1]:
# Activate matplotlib Jupyter widgets
%matplotlib widget

In [2]:
import numpy as np

from py_alf import ALF_source, Simulation              # Interface with ALF

In [3]:
alf_src = ALF_source()

**2.** Create an instance of `Simulation`, setting parameters as desired:

**New:** Proper support for Parallel Tempering

In [6]:
sim = Simulation(
    alf_src,
    "Hubbard",
    [{
        'L1': 4,
        'L2': 4,
        "Nsweep": 1,
        "Nbin": 1000,
        "Lattice_type": "Square",
        "Ham_U": U,
        "mpi_per_parameter_set": 1
    } for U in [1, 2, 3, 4]
    ],
    n_mpi=4,
    #machine='intel'
)

**3.** Compile ALF, downloading it first from the [ALF repository](https://git.physik.uni-wuerzburg.de/ALF/ALF/-/tree/master/) if not found locally. This may take a few minutes:

In [7]:
sim.compile()

Compiling ALF... 
Done.


**4.** Perform the simulation as specified in `sim`:

In [8]:
sim.run()

Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Hubbard_L1=4_L2=4_Square_U=1" for Monte Carlo run.
Create new directory.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Hubbard_L1=4_L2=4_Square_U=1/Temp_0" for Monte Carlo run.
Create new directory.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Hubbard_L1=4_L2=4_Square_U=1/Temp_1" for Monte Carlo run.
Create new directory.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Hubbard_L1=4_L2=4_Square_U=1/Temp_2" for Monte Carlo run.
Create new directory.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Hubbard_L1=4_L2=4_Square_U=1/Temp_3" for Monte Carlo run.
Create new directory.
Run /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out
Error while running /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out.
parameters:


Exception: Error while running /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out.

**New:** Derived observables

In [ ]:
custom_obs = {}

In [ ]:
def obs_squared(obs, sign, N_obs):
    return obs**2 / sign

# Energy squared
custom_obs['E_squared']= {
    'needs': ['Ener_scal'],
    'function': obs_squared,
    'kwargs': {}
}

In [ ]:
def E_pot_kin(E_pot_obs, E_pot_sign, E_pot_N_obs, E_kin_obs, E_kin_sign, E_kin_N_obs):
    return E_pot_obs/E_kin_obs / (E_pot_sign/E_kin_sign)

# Potential Energy / Kinetic Energy
custom_obs['E_pot_kin']= {
    'needs': ['Pot_scal', 'Kin_scal'],
    'function': E_pot_kin,
    'kwargs': {}
}

In [ ]:
def R_k(obs, back, sign, N_orb, N_tau, dtau, latt,
        ks=[(0., 0.)], mat=None, NNs=[(1, 0), (0, 1), (-1, 0), (0, -1)]):
    """RG-invariant quantity derived from a correlatian function.
    
    obs.shape = (N_orb, N_orb, N_tau, latt.N)
    back.shape = (N_orb,)
    """
    if mat is None:
        mat = np.identity(N_orb)
    out = 0
    for k in ks:
        n = latt.k_to_n(k)

        J1 = (obs[..., n].sum(axis=-1) * mat).sum()
        J2 = 0
        for NN in NNs:
            i = latt.nnlistk[n, NN[0], NN[1]]
            J2 += (obs[..., i].sum(axis=-1) * mat).sum() / len(NNs)
        out += (1 - J2/J1)

    return out / len(ks)

# RG-invariant quantity for ferromagnetic order
custom_obs['R_Ferro']= {
    'needs': ['SpinZ_eq'],
    'function': R_k,
    'kwargs': {'ks': [[0., 0.]]}
}

# RG-invariant quantity for antiferromagnetic order
custom_obs['R_AFM']= {
    'needs': ['SpinZ_eq'],
    'function': R_k,
    'kwargs': {'ks': [[np.pi, np.pi]]}
}

**New:** Check warmup and autocorrelation

In [ ]:
sim.check_warmup(
    ['Ener_scal', 'Kin_scal', 'Pot_scal', 'E_pot_kin', 'R_Ferro', 'R_AFM'],
    custom_obs=custom_obs, gui='ipy'
)

In [ ]:
sim.check_rebin(
    ['Ener_scal', 'Kin_scal', 'Pot_scal', 'E_pot_kin', 'R_Ferro', 'R_AFM'],
    custom_obs=custom_obs, gui='ipy'
)

**New:** Lattice symmetries in analysis

In [ ]:
# Define list of transformations (Lattice, i) -> new_i
# Default analysis will average over all listed elements
def sym_c4_0(latt, i): return i
def sym_c4_1(latt, i): return latt.rotate(i, np.pi*0.5)
def sym_c4_2(latt, i): return latt.rotate(i, np.pi)
def sym_c4_3(latt, i): return latt.rotate(i, np.pi*1.5)

sym_c4 = [sym_c4_0, sym_c4_1, sym_c4_2, sym_c4_3]

**5.** Perform anaylsis:

In [ ]:
sim.analysis(symmetry=sym_c4, custom_obs=custom_obs)

**6.** Read analysis results:

In [ ]:
obs = sim.get_obs()

In [ ]:
obs

which are available for further analyses. For instance, the internal energy of the system (and its error) is accessed by:

In [ ]:
obs[['ham_u', 'Ener_scal0', 'Ener_scal0_err', 'Ener_scal_sign', 'E_pot_kin', 'E_pot_kin_err', 'R_Ferro', 'R_AFM']]

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.errorbar(obs.ham_u, obs.E_pot_kin, obs.E_pot_kin_err)

**New:** 2dplot on bravais lattice

In [ ]:
from py_alf import Lattice

In [ ]:
item = obs.iloc[0]

latt = Lattice(item.Den_eq_lattice)

latt.plot_k(item.Den_eqK[0, 0])
latt.plot_r(item.Den_eqR[0, 0])